# VEDA - Run NRT CONUS for 2023

This notebook runs CONUS for the whole year of 2023 from the start of the year at each timestep. It mimics the NRT workflow. See "Performance.ipynb" for analysis.

In [ ]:
# If you haven't installed the fireatlas code yet, uncomment the following line and run this cell.

# !pip install -e .. -q

# After this runs, restart the notebook kernel.

First do some configuring of settings to get the right number of Dask workers and the right S3 path set up.

In [ ]:
import dask
from fireatlas import settings

# turn off dask warnings
dask.config.set({'logging.distributed': 'error'})

settings.S3_PATH = "s3://veda-data-store-staging/EIS"
settings.LOG_FILENAME = "running.log"
settings.N_DASK_WORKERS = 12

In [ ]:
import os
from shutil import rmtree
from time import time

from fireatlas import FireRunDaskCoordinator, preprocess
from fireatlas.FireTime import t_generator, t2d, t2dt

tst = [2023, 1, 1, "AM"]
ted = [2023, 12, 31, "PM"]

region = ("CONUSV3", [-126.401171875,24.071240929282325,-61.36210937500001,49.40003415463647])

Now to mimic what happens in a near real time run, we will iterate throught the whole year and use each half-day timestep as the value for `ted`. NB: This is not the most efficient way to create a year of archive data.

In [ ]:
# with open("/home/jovyan/fireatlas_nrt/notebooks/veda-output.csv", "w") as f:
#     f.write("t,took\n")

# Starting at the place where the algorithm last quit
for ted in t_generator([2023, 3, 15, 'PM'], ted):
    if ted == tst:
        continue
    t_start = time()
    FireRunDaskCoordinator.Run(region, tst, ted)
    t_end = time()
    with open("/home/jovyan/fireatlas_nrt/notebooks/veda-output.csv", "a") as f:
        f.write(f"{t2dt(ted)},{t_end-t_start}\n")

    # clean things out
    for path in [
        f'{settings.get_path("local")}/{settings.PREPROCESSED_DIR}/{region[0]}',
        f'{settings.get_path("local")}/{settings.OUTPUT_DIR}/{region[0]}',
    ]:
        if os.path.exists(path):
            rmtree(path)